# Importing modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox

pd.options.display.max_rows = 50

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
## Install xlrd package to load Excel files
# conda install openpyxl
## conda install xlrd

## Data Cleaning

### loading & exploring Data

In [ ]:
re_df = pd.read_excel('Data_MidTerm_Project_Real_State_Regression.xls')

In [ ]:
re_df.columns

In [ ]:
re_df.dtypes

numerical:
'sqft_living', 
'sqft_lot',
'sqft_living15'
'sqft_lot15'
'sqft_above',
'sqft_basement'
'price'

categorical:
bedrooms
bathrooms
floors in integer
waterfront = categorical (yes/no)   = view?
'condition'
'grade'
'yr_built'
'yr_renovated'  = replace 0 with nan 
'zipcode'
'lat'
'long'


### droping columns

In [ ]:
re_df.drop(['id'],axis=1, inplace=True)
#re_df.drop(['date'],axis=1, inplace=True)
#re_df.drop(['sqft_living'],axis=1, inplace=True)
#re_df.drop(['sqft_lot'],axis=1, inplace=True)
#re_df.drop(['lat'],axis=1, inplace=True)
#re_df.drop(['long'],axis=1, inplace=True)

### correcting data types

In [ ]:
re_df.info()

In [ ]:
#re_df["floors"] = re_df["floors"].astype(int)

### droping nan values

In [ ]:
#re_df.dropna(inplace=True)

### removing duplicates

In [ ]:
#re_df.drop_duplicates(inplace=True)

### Replacing null values with nan

#### years renovated

The yr_renovated column has more than 20.683 values=0. Does that make sense? .....

In [ ]:
re_df.yr_renovated.hist()

In [ ]:
#re_df["yr_renovated"].replace(0, np.nan, inplace=True)

#re_df["view"].replace(0, np.nan, inplace=True)

In [ ]:
re_df.yr_renovated.hist()

#### square feet basement

In [ ]:
re_df.sqft_basement.hist()

In [ ]:
#re_df["sqft_basement"].replace(0, np.nan, inplace=True)
re_df.sqft_basement.hist()

### renaming waterfront column features

In [ ]:
def clean_waterfront(x):
    if x == 0:
        return 'no'
    elif x == 1:
        return 'yes'

In [ ]:
re_df["waterfront"] = list(map(clean_waterfront,re_df["waterfront"])) 

### concatinate lat & long

In [ ]:
re_df['location'] = re_df[['long', 'lat']].apply(tuple, axis=1)
re_df.drop(['lat'],axis=1, inplace=True)
re_df.drop(['long'],axis=1, inplace=True)
re_df

### removing outliers

#### bedrooms

In [ ]:
sns.boxplot(re_df['bedrooms'])

In [ ]:
Q1 = re_df['bedrooms'].quantile(0.25)
Q3 = re_df['bedrooms'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 

filter = (re_df['bedrooms'] >= Q1 - 1.5 * IQR) & (re_df['bedrooms'] <= Q3 + 1.5 *IQR)
re_df = re_df.loc[filter]

In [ ]:
sns.boxplot(re_df['bedrooms'])

530 rows removed

#### bathrooms

In [ ]:
sns.boxplot(re_df['bathrooms'])

In [ ]:
Q1 = re_df['bathrooms'].quantile(0.25)
Q3 = re_df['bathrooms'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 

filter = (re_df['bathrooms'] >= Q1 - 1.5 * IQR) & (re_df['bathrooms'] <= Q3 + 1.5 *IQR)
re_df = re_df.loc[filter]

In [ ]:
sns.boxplot(re_df['bathrooms'])

461 rows removed

#### further exploration

In [ ]:
#re_df = re_df[re_df.sqft_above != 7420]

In [ ]:
#re_df = re_df[re_df.sqft_lot15 != 560617]

## Plot the Correlation Heatmap

In [ ]:
g = sns.PairGrid(re_numerical)
g.map(sns.scatterplot)

## X Y split

In [ ]:
X=re_df.drop("price", axis=1)
y=re_df["price"]

## Data Splitting

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=.30,random_state=123)

In [ ]:
y_train =y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

## Data Transformation

### Hot coding Categorical Variables

In [ ]:
X_train_cat=X_train[['bedrooms','bathrooms','floors','waterfront','condition','grade','yr_built','yr_renovated','zipcode','location']]
X_train_cat

X_test_cat=X_test[['bedrooms','bathrooms','floors','waterfront','condition','grade','yr_built','yr_renovated','zipcode','location']]
X_test_cat

In [ ]:
X_train_cat=pd.get_dummies(X_train_cat, drop_first=True)
pd.DataFrame(OneHotEncoder(drop='first').fit_transform(X_train_cat).toarray(),
columns=OneHotEncoder(drop='first').fit(X_train_cat).get_feature_names(input_features=X_train_cat.columns))

X_test_cat=pd.get_dummies(X_test_cat, drop_first=True)
pd.DataFrame(OneHotEncoder(drop='first').fit_transform(X_test_cat).toarray(),
columns=OneHotEncoder(drop='first').fit(X_test_cat).get_feature_names(input_features=X_test_cat.columns))

In [ ]:
X_train_cat = X_train_cat.reset_index(drop=True)
X_test_cat = X_test_cat.reset_index(drop=True)

### Standardize numerical

In [ ]:
X_train_num=X_train[['sqft_living','sqft_lot','sqft_living15','sqft_lot15','sqft_above','sqft_basement']]
X_train_num

X_test_num=X_test[['sqft_living','sqft_lot','sqft_living15','sqft_lot15','sqft_above','sqft_basement']]
X_test_num

In [ ]:
std_scaler=StandardScaler().fit(X_train_num)   ## finding the parameters (mean, variance from the training set )

X_train_scaled=std_scaler.transform(X_train_num)

In [ ]:
X_test_scaled=std_scaler.transform(X_test_num)

In [ ]:
print(X_train_scaled)
print("--------")
print(X_test_scaled)

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns= X_train_num.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns= X_test_num.columns)

In [ ]:
X_train_scaled = X_train_scaled.reset_index(drop=True)
X_test_scaled = X_test_scaled.reset_index(drop=True)

## Concatination 

In [ ]:
X_train_con=pd.concat([X_train_scaled,X_train_cat],axis=1)
X_test_con=pd.concat([X_test_scaled,X_test_cat],axis=1)

In [ ]:
X_train_con

##  Apply Linear Regression

### Modelling using Statsmodels

In [ ]:
X_train_const_scaled = sm.add_constant(X_train_con) # adding a constant

model = sm.OLS(y_train, X_train_const_scaled).fit()
predictions_train = model.predict(X_train_const_scaled) 

X_test_const_scaled = sm.add_constant(X_test_con) # adding a constant
predictions_test = model.predict(X_test_const_scaled) 
print_model = model.summary()
print(print_model)

In [ ]:
predictions_test

In [ ]:
#display adjusted R-squared
print(model.rsquared_adj)

### Modeling using Scikit Learn

In [ ]:
model=LinearRegression()    # model
model.fit(X_train_con, y_train)   # model train

In [ ]:
model.coef_

In [ ]:
model.intercept_

Making prediction

In [ ]:
y_pred=model.predict(X_test_con)   # model prediction
##y_pred_train=model.predict(X_train_scaled)

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
result=pd.DataFrame({"y_test":y_test,"y_pred":y_pred})
result

Linear Regression Train-test split. Standardize the data (after the data split). Apply linear regression. Model Interpretation. Model Validation Model Evaluation: MSE. RMSE. MAE. R2. Adjusted R2. Feature Importance.

In [ ]:
# Make an scatter plot y_pred vs y
# What kind of plot you will get if all the all the predictions are ok?
# A stright line

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1].hist(y_test - y_pred)
ax[1].set_xlabel("Test y-y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred,y_test - y_pred,"o")
ax[2].set_xlabel("predited")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred,np.zeros(len(y_pred)),linestyle='dashed')

#### Error metrics

In [ ]:
print(mse(y_test,y_pred))
print(mae(y_test,y_pred))

In [ ]:
R2=r2_score(y_test,y_pred)
R2

In [ ]:
X_test.shape[1]

In [ ]:
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
Adj_R2

#### Feature Importance

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train_con.columns,
    'Importance': abs(model.coef_)
})
features_importances = features_importances.sort_values(by='Importance')

In [ ]:
features_importances

In [ ]:
plt.bar(x=features_importances['Attribute'], height=features_importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

#### Evaluating predictions

In [ ]:
# MAE: mean absolute error
MAE = np.mean( np.abs( y_test - y_pred ) )
MAE

In [ ]:
# MSE: mean squared error
MSE = np.mean( np.power( y_test - y_pred , 2 ) )
MSE

In [ ]:
# RMSE: root mean squared error
RMSE = np.sqrt(MSE)
RMSE

In [ ]:
X.hist(figsize=(10,10))